In [3]:
import numpy as np
import neal, math, dimod

In [4]:
#Code to generate a circulant matrix given the first row
def generate_codematrix(row):
  n = len(row)
  #Create a matrix filled with zeros
  circ_matrix = np.zeros((n, n), dtype=row.dtype)
  # Fill the matrix using the first row
  for i in range(n):
      circ_matrix[i, :] = np.roll(row, i)
  return circ_matrix

In [5]:
#input code info as (first row of B matrix), known_code_distance_if_available
#self-dual code samples from grassl database
code_rows=[(np.array([0, 0, 1, 1, 1, 0]),4),
          (np.array([0, 0, 0, 1, 1, 0, 0]),3),
          (np.array([0, 1, 0, 0, 1, 0, 0, 1 ]),4)]
        #(np.array([0, 0, 0, 1, 1, 1, 1, 0, 0 ]),4),
         #(np.array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1]),4),
          #(np.array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]),4),
          #(np.array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0]),12,6),
          #(np.array([0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0 ]),13,5),
          #(np.array([0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1 ]),14,6),
          #(np.array([0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1]),15,6)]

In [6]:
#generate the full circulant generator matrix
code_matrices = [generate_codematrix(x[0]) for x in code_rows]

In [7]:
def create_qubo(code_matrix):
  B=code_matrix
  n=int(len(B));
  r= math.ceil(math.log(1+math.floor(np.count_nonzero(B[0])/2))/math.log(2))
  d=n+n*r; #number of primary + auxillary variables
  I=np.identity(n);
  Bsq= np.dot(B,B);
  qubo_mat=np.zeros((d,d))

  # Fill out primary-variables block
  qubo_mat[:n,:n] = I+Bsq-B;

  # Fill out first n rows and n columns (cross-terms between primary variables and auxillary variables)
  for i in range(1,r+1):
    qubo_mat[:n,i*n:(i+1)*n]= 2**(i-1)* (I -2*B)
    qubo_mat[i*n:(i+1)*n,:n]= 2**(i-1)* (I -2*B)

  #Fill Auxillary variable block:
  multiplicative_factor_matrix = np.array([[2**(i + j) for j in range(r)] for i in range(r)]);
  qubo_mat[n:,n:] = np.kron(multiplicative_factor_matrix,4*I)

  #create dictionary that the sampler uses to represent Binary quadratic models
  linear_dict={}
  quadratic_dict={}
  offset = qubo_mat[0,0] #We set the first variable to 1 to deal with constraint
  for i in range(1,d):
    linear_dict[i] =  qubo_mat[i,i]  + 2*qubo_mat[0,i] #accounting to contribution from first variable
    for j in range(i+1,d):
      quadratic_dict[(i,j)] = 2*qubo_mat[i,j] #the 2 accounts for both terms coming from the symmetric matrix M

  bqm = dimod.BinaryQuadraticModel(linear_dict,quadratic_dict, offset, dimod.BINARY);
  return bqm

In [8]:
#create the QUBO matrix including the auxillary variables
code_qubos = [create_qubo(x) for x in code_matrices]

In [9]:
#Run neal simulated annealer with default settings
sampler=neal.SimulatedAnnealingSampler()
samples = [sampler.sample(x) for x in code_qubos]

#exact solver for code distance for small enough codes
#exact_samples = [dimod.ExactSolver().sample(x)for x in code_qubos]


In [10]:
for x in samples:
  print("estimated code distance is:",x.lowest().first.energy)

estimated code distance is: 4.0
estimated code distance is: 3.0
estimated code distance is: 4.0
